In [4]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-

from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os


class GAN:
    def __init__(self):
        # 图像尺寸
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # 优化器，使用Adam算法优化
        optimizer = Adam(0.0002, 0.5)

        # 生成器
        self.generator = self.build_generator()

        # 判别器
        self.discriminator = self.build_discriminator()
        # 判别器编译
        self.discriminator.compile(loss="binary_crossentropy",
                                   optimizer=optimizer,
                                   metrics=["accuracy"])

        # z是噪声，送入生成器生成图像
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # 固定判别器，训练生成器
        self.discriminator.trainable = False

        # 将生成器生成的图像传入判别器判断
        validity = self.discriminator(img)

        # 结合生成器和判别器
        self.combined = Model(z, validity)
        self.combined.compile(loss="binary_crossentropy", optimizer=optimizer)

    def build_generator(self):
        # 序列模型
        model = Sequential()

        model.add(Dense(128 * 16 * 16, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((16, 16, 128)))
        # 上采样，将图像放大一倍
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(32, kernel_size=4, padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(16, kernel_size=4, padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(8, kernel_size=4, padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(3, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential()

        model.add(Conv2D(32, kernel_size=4, input_shape=(64, 64, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        # model.add(ZeroPadding2D())
        model.add(Activation("relu"))
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(256, kernel_size=4, padding="same"))
        model.add(Activation("relu"))
        model.add(Flatten())
        model.add(Dense(1, activation="sigmoid"))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):
        # 读取图片数据
        x_train = load_data()
        # 归一化
        x_train = x_train / 127.5 - 1.

        # 图像数据的判别值为1
        valid = np.ones((batch_size, 1))
        # 生成图像的判别值为0
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            # ---------------------
            #  Train Discriminator
            # ---------------------
            # 随机选取一个batch的图像送入训练
            idx = np.random.randint(0, x_train.shape[0], batch_size)
            images = x_train[idx]

            # 产生噪声，生成图像
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_images = self.generator.predict(noise)

            # 训练判别器
            d_loss_real = self.discriminator.train_on_batch(images, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_images, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, valid)

            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))

            # 保存图像
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    # 保存生成结果
    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, :])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()


# 读取extra_data中的图片，并保存
def load_data():
    x_train = np.empty((36740, 64, 64, 3), dtype="float32")
    images = os.listdir('./extra_data/images/')

    lens = len(images)
    for i in range(lens):
        img = Image.open('./extra_data/images/' + images[i])
        arr = np.asarray(img, dtype="float32")
        x_train[i, :, :, :] = arr

    return x_train


def main():
    gan = GAN()
    gan.train(epochs=10001, batch_size=32, sample_interval=20)


if __name__ == "__main__":
    main()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 32768)             3309568   
_________________________________________________________________
reshape_4 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
up_sampling2d_13 (UpSampling (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 32, 32, 128)       262272    
_________________________________________________________________
activation_37 (Activation)   (None, 32, 32, 128)       0         
_________________________________________________________________
up_sampling2d_14 (UpSampling (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 64, 64, 64)        131136    
__________

74 [D loss: 0.309547, acc.: 96.88%] [G loss: 3.399246]
75 [D loss: 0.021823, acc.: 100.00%] [G loss: 4.458967]
76 [D loss: 0.004119, acc.: 100.00%] [G loss: 5.405380]
77 [D loss: 0.002373, acc.: 100.00%] [G loss: 5.654681]
78 [D loss: 0.002714, acc.: 100.00%] [G loss: 5.482236]
79 [D loss: 0.002532, acc.: 100.00%] [G loss: 5.382257]
80 [D loss: 0.002550, acc.: 100.00%] [G loss: 5.385917]
81 [D loss: 0.002472, acc.: 100.00%] [G loss: 5.441259]
82 [D loss: 0.002421, acc.: 100.00%] [G loss: 5.499623]
83 [D loss: 0.016191, acc.: 98.44%] [G loss: 4.902355]
84 [D loss: 0.005555, acc.: 100.00%] [G loss: 4.696953]
85 [D loss: 0.007457, acc.: 100.00%] [G loss: 4.580229]
86 [D loss: 0.011264, acc.: 100.00%] [G loss: 4.810782]
87 [D loss: 0.008186, acc.: 100.00%] [G loss: 5.593246]
88 [D loss: 0.057399, acc.: 96.88%] [G loss: 10.103336]
89 [D loss: 0.000008, acc.: 100.00%] [G loss: 12.773098]
90 [D loss: 0.031879, acc.: 98.44%] [G loss: 10.140446]
91 [D loss: 0.000370, acc.: 100.00%] [G loss: 7.7

220 [D loss: 0.246774, acc.: 90.62%] [G loss: 6.911319]
221 [D loss: 0.716751, acc.: 67.19%] [G loss: 2.211820]
222 [D loss: 0.354519, acc.: 81.25%] [G loss: 4.269082]
223 [D loss: 0.219182, acc.: 92.19%] [G loss: 4.161921]
224 [D loss: 0.311697, acc.: 90.62%] [G loss: 3.023564]
225 [D loss: 0.136305, acc.: 96.88%] [G loss: 3.586969]
226 [D loss: 0.166567, acc.: 96.88%] [G loss: 3.618485]
227 [D loss: 0.195986, acc.: 93.75%] [G loss: 4.410527]
228 [D loss: 0.169123, acc.: 95.31%] [G loss: 4.459849]
229 [D loss: 0.310596, acc.: 92.19%] [G loss: 4.577859]
230 [D loss: 0.225139, acc.: 95.31%] [G loss: 5.434299]
231 [D loss: 0.199817, acc.: 92.19%] [G loss: 5.219582]
232 [D loss: 1.857952, acc.: 40.62%] [G loss: 12.363375]
233 [D loss: 1.597691, acc.: 67.19%] [G loss: 4.452488]
234 [D loss: 0.715363, acc.: 70.31%] [G loss: 2.581271]
235 [D loss: 0.299408, acc.: 89.06%] [G loss: 2.794814]
236 [D loss: 0.346719, acc.: 82.81%] [G loss: 2.749320]
237 [D loss: 0.323619, acc.: 87.50%] [G loss: 2

367 [D loss: 0.185036, acc.: 93.75%] [G loss: 3.715505]
368 [D loss: 0.088160, acc.: 100.00%] [G loss: 3.224242]
369 [D loss: 0.111154, acc.: 98.44%] [G loss: 3.718829]
370 [D loss: 0.055390, acc.: 100.00%] [G loss: 4.449441]
371 [D loss: 0.055661, acc.: 98.44%] [G loss: 3.711266]
372 [D loss: 0.158079, acc.: 96.88%] [G loss: 4.005573]
373 [D loss: 0.124700, acc.: 96.88%] [G loss: 5.177376]
374 [D loss: 0.462551, acc.: 78.12%] [G loss: 6.936359]
375 [D loss: 0.125664, acc.: 96.88%] [G loss: 5.050896]
376 [D loss: 0.237652, acc.: 93.75%] [G loss: 5.480180]
377 [D loss: 0.109805, acc.: 95.31%] [G loss: 4.847604]
378 [D loss: 0.314087, acc.: 84.38%] [G loss: 5.038310]
379 [D loss: 0.107568, acc.: 96.88%] [G loss: 4.829476]
380 [D loss: 0.374359, acc.: 89.06%] [G loss: 5.365805]
381 [D loss: 0.108830, acc.: 95.31%] [G loss: 4.870530]
382 [D loss: 0.068346, acc.: 100.00%] [G loss: 4.833345]
383 [D loss: 0.209229, acc.: 93.75%] [G loss: 4.762688]
384 [D loss: 0.222835, acc.: 90.62%] [G loss:

514 [D loss: 0.159633, acc.: 96.88%] [G loss: 3.709929]
515 [D loss: 0.120126, acc.: 96.88%] [G loss: 3.448215]
516 [D loss: 0.159345, acc.: 95.31%] [G loss: 4.488483]
517 [D loss: 0.372849, acc.: 84.38%] [G loss: 4.295206]
518 [D loss: 0.242931, acc.: 90.62%] [G loss: 3.412634]
519 [D loss: 0.324220, acc.: 90.62%] [G loss: 8.251400]
520 [D loss: 0.186054, acc.: 92.19%] [G loss: 4.549349]
521 [D loss: 1.213286, acc.: 50.00%] [G loss: 12.320028]
522 [D loss: 1.132940, acc.: 68.75%] [G loss: 3.224218]
523 [D loss: 0.476838, acc.: 76.56%] [G loss: 3.720609]
524 [D loss: 0.304596, acc.: 87.50%] [G loss: 2.951128]
525 [D loss: 0.570488, acc.: 75.00%] [G loss: 2.902273]
526 [D loss: 0.440672, acc.: 81.25%] [G loss: 2.761114]
527 [D loss: 0.199882, acc.: 95.31%] [G loss: 2.732292]
528 [D loss: 0.423979, acc.: 81.25%] [G loss: 2.088376]
529 [D loss: 0.231698, acc.: 96.88%] [G loss: 2.530381]
530 [D loss: 0.222091, acc.: 95.31%] [G loss: 2.850934]
531 [D loss: 0.306803, acc.: 95.31%] [G loss: 2

...

9291 [D loss: 0.047073, acc.: 98.44%] [G loss: 5.968147]
9292 [D loss: 0.060656, acc.: 98.44%] [G loss: 6.272763]
9293 [D loss: 0.024652, acc.: 100.00%] [G loss: 6.404282]
9294 [D loss: 0.023810, acc.: 100.00%] [G loss: 7.341425]
9295 [D loss: 0.224756, acc.: 89.06%] [G loss: 7.381175]
9296 [D loss: 0.103698, acc.: 95.31%] [G loss: 5.148566]
9297 [D loss: 0.017934, acc.: 100.00%] [G loss: 5.954864]
9298 [D loss: 0.045752, acc.: 96.88%] [G loss: 7.464694]
9299 [D loss: 0.040932, acc.: 98.44%] [G loss: 5.674924]
9300 [D loss: 0.024868, acc.: 100.00%] [G loss: 5.899009]
9301 [D loss: 0.117584, acc.: 93.75%] [G loss: 10.881145]
9302 [D loss: 0.467747, acc.: 76.56%] [G loss: 8.123086]
9303 [D loss: 0.103371, acc.: 93.75%] [G loss: 5.258514]
9304 [D loss: 0.231778, acc.: 87.50%] [G loss: 6.703971]
9305 [D loss: 0.031277, acc.: 98.44%] [G loss: 7.785790]
9306 [D loss: 1.363573, acc.: 40.62%] [G loss: 11.257100]
9307 [D loss: 2.621475, acc.: 12.50%] [G loss: 4.414816]
9308 [D loss: 1.147351, a

9434 [D loss: 0.023251, acc.: 100.00%] [G loss: 5.596378]
9435 [D loss: 0.010940, acc.: 100.00%] [G loss: 5.548928]
9436 [D loss: 0.049006, acc.: 98.44%] [G loss: 6.998593]
9437 [D loss: 0.030744, acc.: 100.00%] [G loss: 6.152413]
9438 [D loss: 0.085318, acc.: 96.88%] [G loss: 7.141030]
9439 [D loss: 0.096551, acc.: 95.31%] [G loss: 4.348750]
9440 [D loss: 0.040055, acc.: 100.00%] [G loss: 6.444351]
9441 [D loss: 0.074325, acc.: 96.88%] [G loss: 4.768976]
9442 [D loss: 0.131663, acc.: 96.88%] [G loss: 8.756787]
9443 [D loss: 0.094787, acc.: 96.88%] [G loss: 6.901450]
9444 [D loss: 0.038586, acc.: 98.44%] [G loss: 6.672620]
9445 [D loss: 0.005927, acc.: 100.00%] [G loss: 5.967930]
9446 [D loss: 0.118565, acc.: 96.88%] [G loss: 7.648143]
9447 [D loss: 0.088258, acc.: 98.44%] [G loss: 5.143156]
9448 [D loss: 0.030781, acc.: 100.00%] [G loss: 6.199489]
9449 [D loss: 0.008732, acc.: 100.00%] [G loss: 6.912700]
9450 [D loss: 0.031620, acc.: 98.44%] [G loss: 5.204663]
9451 [D loss: 0.079419, 

9577 [D loss: 3.086232, acc.: 50.00%] [G loss: 2.480731]
9578 [D loss: 0.348962, acc.: 85.94%] [G loss: 2.700796]
9579 [D loss: 0.403390, acc.: 78.12%] [G loss: 3.116062]
9580 [D loss: 0.664189, acc.: 70.31%] [G loss: 3.059839]
9581 [D loss: 0.509612, acc.: 75.00%] [G loss: 1.985518]
9582 [D loss: 0.438847, acc.: 76.56%] [G loss: 2.345985]
9583 [D loss: 0.245119, acc.: 89.06%] [G loss: 3.991522]
9584 [D loss: 0.523153, acc.: 73.44%] [G loss: 2.624271]
9585 [D loss: 0.351204, acc.: 85.94%] [G loss: 3.646924]
9586 [D loss: 0.363469, acc.: 84.38%] [G loss: 3.601473]
9587 [D loss: 0.464283, acc.: 76.56%] [G loss: 2.234865]
9588 [D loss: 0.383827, acc.: 82.81%] [G loss: 3.432706]
9589 [D loss: 0.420810, acc.: 78.12%] [G loss: 2.619977]
9590 [D loss: 0.277811, acc.: 89.06%] [G loss: 3.412462]
9591 [D loss: 0.151298, acc.: 100.00%] [G loss: 2.911395]
9592 [D loss: 0.196022, acc.: 90.62%] [G loss: 3.169986]
9593 [D loss: 0.161748, acc.: 95.31%] [G loss: 4.007856]
9594 [D loss: 0.285232, acc.: 

9721 [D loss: 0.057174, acc.: 98.44%] [G loss: 6.434076]
9722 [D loss: 0.020810, acc.: 100.00%] [G loss: 6.507601]
9723 [D loss: 0.029783, acc.: 98.44%] [G loss: 7.773334]
9724 [D loss: 0.026334, acc.: 100.00%] [G loss: 6.256697]
9725 [D loss: 0.044678, acc.: 98.44%] [G loss: 5.871766]
9726 [D loss: 0.010169, acc.: 100.00%] [G loss: 6.559139]
9727 [D loss: 0.039431, acc.: 100.00%] [G loss: 5.851992]
9728 [D loss: 0.006172, acc.: 100.00%] [G loss: 6.048547]
9729 [D loss: 0.043927, acc.: 98.44%] [G loss: 4.486745]
9730 [D loss: 0.040887, acc.: 100.00%] [G loss: 6.569571]
9731 [D loss: 0.024168, acc.: 100.00%] [G loss: 8.090274]
9732 [D loss: 0.017351, acc.: 100.00%] [G loss: 7.580727]
9733 [D loss: 0.012373, acc.: 100.00%] [G loss: 6.850964]
9734 [D loss: 0.021948, acc.: 98.44%] [G loss: 6.470300]
9735 [D loss: 0.008243, acc.: 100.00%] [G loss: 6.012383]
9736 [D loss: 0.015105, acc.: 100.00%] [G loss: 5.595576]
9737 [D loss: 0.040446, acc.: 98.44%] [G loss: 5.014244]
9738 [D loss: 0.1254

9864 [D loss: 0.010682, acc.: 100.00%] [G loss: 6.970066]
9865 [D loss: 0.063940, acc.: 96.88%] [G loss: 8.144466]
9866 [D loss: 0.021846, acc.: 100.00%] [G loss: 8.252180]
9867 [D loss: 0.030914, acc.: 98.44%] [G loss: 6.071943]
9868 [D loss: 0.056119, acc.: 96.88%] [G loss: 6.744749]
9869 [D loss: 0.015939, acc.: 100.00%] [G loss: 6.857948]
9870 [D loss: 0.045151, acc.: 98.44%] [G loss: 6.755129]
9871 [D loss: 0.013689, acc.: 100.00%] [G loss: 7.334470]
9872 [D loss: 0.083865, acc.: 93.75%] [G loss: 6.505020]
9873 [D loss: 0.052084, acc.: 98.44%] [G loss: 6.732705]
9874 [D loss: 0.030933, acc.: 100.00%] [G loss: 7.876648]
9875 [D loss: 0.051777, acc.: 98.44%] [G loss: 8.438483]
9876 [D loss: 0.051176, acc.: 98.44%] [G loss: 4.797068]
9877 [D loss: 0.048847, acc.: 100.00%] [G loss: 6.536644]
9878 [D loss: 0.031841, acc.: 100.00%] [G loss: 5.993201]
9879 [D loss: 0.014403, acc.: 100.00%] [G loss: 5.260617]
9880 [D loss: 0.029129, acc.: 100.00%] [G loss: 7.139126]
9881 [D loss: 0.068387